# Pengambilan Data

In [22]:
topik = 'cryptocurrency'
waktu_ambil = '14 maret malam'

In [23]:
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [24]:
import twitter_credentials
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import string
import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

In [26]:
# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth

In [27]:
# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)

In [28]:
# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
      
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                       tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)

In [29]:
 class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
         
    def clean_tweet(self, tweet):
    
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w +:\/\/ \S +)", " ", tweet).split())
    
    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        # analysis = analysis.translate(from_lang = 'in',to='en')

        if analysis.sentiment.polarity > 0:
            return "Positif"
        elif analysis.sentiment.polarity == 0:
            return "Netral"
        else:
            return "Negatif"
        
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])
        
        df['id'] = np.array([tweet.id_str for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['lang'] = np.array([tweet.lang for tweet in tweets])
        df['place'] = np.array([tweet.place for tweet in tweets ])
        

        return df

In [30]:
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()
    keyword = [topik]
    api = twitter_client.get_twitter_client_api()
    tweets = api.search(keyword, count=100, lang = 'en')

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])

    print(df)

                                               tweets                   id  \
0   2 Months Ago, Andreas Antonopoulos Explained W...  1238815184648638466   
1   People’s Bank Of China Making Progress Toward ...  1238815143229673472   
2   RT @WinPlayApp: Read how #Blockchain restricti...  1238815088599011329   
3   @jacindaardern Biggest NZ cryptocurrency theft...  1238815052162990080   
4   RT @AtomarsExchange: Round 2 of the project vo...  1238815015253291008   
5   😯🤓Cryptocurrency beneficial for an E-commerce ...  1238814993362989056   
6   RT @iqbal39291461: Centric public sale is now ...  1238814983959519232   
7   RT @crystaljdt15: Free 10000 btc satoshis (tra...  1238814975076032514   
8   RT @CryptoPolka: #BitcoinCash is down 2.17% fo...  1238814970302906368   
9   @TheCryptoDog hm... when we have best time for...  1238814960526012421   
10  RT @BTCHistory: Today in BTC history: \n------...  1238814948622508033   
11  RT @WinPlayApp: Read how #Blockchain restricti...  123881491

# Preproses

In [31]:
def remove_punct(text):
  
    #hapus simbol
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    text = re.sub('rt @[^\s]+',' ', text)
    text = re.sub('@[^\s]+',' ',text)
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    #Remove additional white spaces
    text = re.sub('[\s]+', ' ', text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    #trim
    text = text.strip('\'"')
    text  = "".join([char for char in text if char not in string.punctuation])
    #text = re.sub('[0-9]+', '', text)

    return text

df['tweet_bersih'] = df['tweets'].apply(lambda x: remove_punct(x.lower()))

In [32]:
#--- REMOVE DUPLICATES
df = df.drop_duplicates(['tweet_bersih'])
df

,tweets,id,date,source,lang,place,sentiment,tweet_bersih
0,"2 Months Ago, Andreas Antonopoulos Explained W...",1238815184648638466,2020-03-14 13:12:10,bitcoinagile,en,None,Netral,2 months ago andreas antonopoulos explained wh...
1,People’s Bank Of China Making Progress Toward ...,1238815143229673472,2020-03-14 13:12:00,drumup.io,en,None,Netral,people s bank of china making progress toward ...
2,RT @WinPlayApp: Read how #Blockchain restricti...,1238815088599011329,2020-03-14 13:11:47,Twitter Web App,en,None,Negatif,read how blockchain restriction measures are ...
3,@jacindaardern Biggest NZ cryptocurrency theft...,1238815052162990080,2020-03-14 13:11:39,Twitter Web App,en,None,Netral,biggest nz cryptocurrency theft on your watch...
4,RT @AtomarsExchange: Round 2 of the project vo...,1238815015253291008,2020-03-14 13:11:30,Twitter Web App,en,None,Negatif,round 2 of the project voting competition wil...
5,😯🤓Cryptocurrency beneficial for an E-commerce ...,1238814993362989056,2020-03-14 13:11:25,Twitter Web App,en,None,Netral,cryptocurrency beneficial for an ecommerce st...
6,RT @iqbal39291461: Centric public sale is now ...,1238814983959519232,2020-03-14 13:11:22,Alex Blockchain Bot,en,None,Positif,centric public sale is now open visit and buy...
7,RT @crystaljdt15: Free 10000 btc satoshis (tra...,1238814975076032514,2020-03-14 13:11:20,Alex Blockchain Bot,en,None,Positif,free 10000 btc satoshis tradable claim it bef...
8,RT @CryptoPolka: #BitcoinCash is down 2.17% fo...,1238814970302906368,2020-03-14 13:11:19,arjiebharjie,en,None,Negatif,bitcoincash is down 217 for the day bch is cu...
9,@TheCryptoDog hm... when we have best time for...,1238814960526012421,2020-03-14 13:11:17,Twitter for iPhone,en,None,Positif,hm when we have best time for cryptocurrency ...


In [33]:
df.to_excel("Data Tanggal " + waktu_ambil + " (" + topik + ").xlsx")
df.to_csv("Data Tanggal " + waktu_ambil + " (" + topik + ").csv")